In [1]:
from qcmdpc import *

**Entrée:**
    k: dimension du vecteur
    t: poids de Hamming du vecteur
    **Sortie:** vecteur aléatoire du poids de Hamming t et de la dimension k

In [2]:
def genRandomVector(k, t):
   
    randomVector = np.zeros(k, dtype = np.int32)

    # choisir t positions aléatoires parmi k positions
    randomPositions = np.random.choice(k, t, replace=False)

    # attribuer aux positions aléatoires la valeur 1
    for j in randomPositions:
        randomVector[j] = 1
    
    return randomVector

**Entrée:**
    G: matrice génératrice d'une matrice QC-MDPC
    m: texte en clair
    e: vecteur d'erreur
    **Sortie:**
    texte chiffré: message chiffré

In [3]:
def encryptMcEliece(G, m, e):
    rows, cols = G.shape
    n = cols
    r = cols - rows
    
    #le chiffrement suit le document de recherche, texte chiffré = m * G + e
    ciphertext = np.copy(np.add(np.matmul(m, G), e))
    
    ciphertext = convertBinary(ciphertext)

    #Décommentez ceci pour vérifier si le cryptage est correct
    #print("Plaintext : ", convertBinary(np.matmul(m, G)))
    #print("Error     : ", randomError)
    #print("Ciphertext: ", ciphertext)
    
    return ciphertext

**Entrée:**
    H: matrice de contrôle de parité (pas nécessairement QC-MDPC)
    c: mot à décoder
    N: coupure pour le nombre d'itérations de retournement de bits
    **Sortie:**
    si le décodage est réussi, retourne le mot décodé
    sinon retourne 0

In [4]:
def bitFlipping(H, c, N):

    print("\nStarting Bit-Flipping Algorithm...")
    rows, cols = H.shape
    
    #Le graphe de Tanner de la matrice de contrôle de parité est représenté à l'aide de listes adjancency
    bitsAdjList = [[] for i in range(cols)]
    checksAdjList = [[] for i in range(rows)]
    
    #nœuds de bits et nœuds de contrôle, et initialisation d'autres paramètres
    bits = np.copy(c)
    checks = np.zeros(rows)
    t = 0           #non. de tours
    numOnes = 0     #compte non. de uns dans les nœuds de contrôle, si numOnes = 0, le mot de code est décodé
    flipOrNot = 0   #utilisera le vote à la majorité pour décider si un morceau doit être retourné
    checkSum = 0    #juste une variable intermédiaire pour le calcul
    
    for i in range(rows):
        for j in range(cols):
            if H[i, j] == 1:
                bitsAdjList[j].append(i)
                checksAdjList[i].append(j)
    
    while t < N:
        #print("Bit-Flipping Decoding Round", t, ":")
        for i in range(rows):
            for j in checksAdjList[i]:
                checkSum += bits[j]
            checks[i] = checkSum % 2
            checkSum = 0
            if checks[i] == 1:
                numOnes += 1

        if numOnes == 0:
            break
        else:
            numOnes = 0
        
        for i in range(cols):
            for j in bitsAdjList[i]:
                if checks[j] == 1:
                    flipOrNot += 1
            if 2*flipOrNot > len(bitsAdjList[i]):
                bits[i] = (bits[i] + 1) % 2
            flipOrNot = 0

        t += 1
                
    if t < N:
        print("Decoded in", t, "step(s).")
        
        return bits
    else:
        print("Cannot decode")
    
    return 0

**Entrée:**
    H: matrice de contrôle de parité (pas nécessairement QC-MDPC)
    y: mot à décoder
    N: seuil pour le nombre d'itérations de produit somme
    p: probabilité qu'un bit soit le chiffre 0
    **Sortie:**
    si le décodage est réussi, retourne le mot décodé
    sinon retourne 0

In [5]:
def sumProduct(H, y, N, p):
    
    print("\nStarting Sum-Product Algorithm...")
    rows, cols = H.shape
    
    #initialiser les variables
    t = 0
    checkSum = 0
    numOnes = 0
    prodTerm = 1

    #initialiser les tableaux et les matrices
    bitsAdjList = [[] for i in range(cols)]
    checksAdjList = [[] for i in range(rows)]
    estimate = np.zeros(2)
    bits = np.zeros(rows, dtype = np.int32)
    checks = np.zeros(rows, dtype = np.int32)
    M = np.zeros((rows, cols))
    E = np.zeros((rows, cols))
    r = np.zeros(cols)
    L = np.zeros(cols)
    x = np.zeros(cols, dtype = np.int32)
    
    estimate = [math.log((1-p)/p), math.log(p/(1-p))]   #L(C_j|Y_j=y_j)
    
    #configurer le graphe de Tanner avec des listes adjancency 
    for i in range(rows):
        for j in range(cols):
            if H[i, j] == 1:
                bitsAdjList[j].append(i)
                checksAdjList[i].append(j)

    #Step 1: vérifier si y est un mot de code avec 1 tour de bit-flipping
    for i in range(rows):
        for j in checksAdjList[i]:
            checkSum += y[j]
        if (checkSum % 2) == 1:
            numOnes += 1
        checkSum = 0
        
    if numOnes == 0:
        return y
        
    checkSum = 0
    numOnes = 0

    #Step 2
    for j in range(cols):
        r[j] = estimate[int(y[j])]
        for i in bitsAdjList[j]:
            M[i,j] = r[j]
    while (t < N):
        t += 1
        
        #Step 3: calculer E [i, j]

        #print("M:", "\n", M, "\n")
        for i in range(rows):
            prodTerm = 1
            for j in checksAdjList[i]:
                prodTerm *= np.tanh(0.5*M[i,j])
            for j in checksAdjList[i]:
                divisor = np.tanh(0.5*M[i,j])
                try:
                    E[i,j] = math.log((1+(prodTerm/divisor))/(1-(prodTerm/divisor)))
                except ValueError:
                    print("Cannot compute E[i,j]")
                    return -1
        #print("E:", "\n", E, "\n")
        
        #Step 4:
        L = np.copy(r)
        #L = L * (N-t) / N
            
        for j in range(cols):
            for s in bitsAdjList[j]:
                L[j] += E[s,j]
            if L[j] > 0:
                x[j] = 0
            elif L[j] < 0:
                x[j] = 1
        #print("L:\n", L)
        
        #Step 4: vérifier si x est un mot de code avec 1 tour de bit-flipping   
        checkSum = 0
        numOnes = 0
        for i in range(rows):
            for j in checksAdjList[i]:
                checkSum += x[j]
            if (checkSum % 2) == 1:
                numOnes += 1
            checkSum = 0
        if numOnes == 0:
            print("Decoded in", t, "step(s).")
            return x

        #Step 5:
        for j in range(cols):
            for i in bitsAdjList[j]:
                M[i,j] = L[j] - E[i,j]

    print("Cannot decode")
    return 0

**Entrée:**
        H: matrice QC-MDPC
        y: texte chiffré
        méthode: soit 'BF' ou 'SP', représentant Bit-Flipping et Sum-Product resp.
        N: coupure pour non. des itérations de décodage
        p: probabilité d'erreur (uniquement pour method = 'SP'. Si method = 'BF',
        peu importe la valeur de p
    **Sortie:**
        decryptedText: texte déchiffré
        (le texte décrypté ne peut être qu'un entier si le décryptage échoue)
    bitFlipping renvoie 0 si le décodage échoue en raison d'un dépassement de l'itération max
    SumProduct renvoie 0 si le décodage échoue en raison d'un dépassement de l'itération maximale
    SumProduct renvoie -1 si le décodage échoue en raison d'une erreur de calcul E [i, j]

In [6]:
def decryptMcEliece(H, y, method, N, p):
  
    r, n = H.shape

    if (method == 'BF'):
        #Décryptage
        decryptedText = bitFlipping(H, y, N)
        
        if type(decryptedText) == int:
            print("Cannot decode by Bit-Flipping algorithm")
        else :
            decryptedText = decryptedText[0: n - r]
            
    elif (method == 'SP'):
        #Décryptage
        decryptedText = sumProduct(H, y, N, p)
        
        if type(decryptedText) == int:
            print("Cannot decode by Sum-Product algorithm")
        else :
            decryptedText = decryptedText[0: n - r]
            
    return decryptedText


**Entrée:**
    texte brut: le message d'origine
    decryptedText: texte déchiffré
    **Sortie:** retourne true si (texte clair == decryptedText) élément par élément, retourne false sinon

In [7]:
def decryptSuccess(plaintext, decryptedText):

    status = np.array_equal(plaintext, decryptedText)
    if (status == True):
        print("Decryption success!")
    else:
        print("Decryption failure!")
        
    return status

Démo simple pour l'algorithme Bit-Flipping
    **Entrée:** matrice de contrôle de parité H, texte chiffré y
    **Sortie:** texte déchiffré y '
    Hypothèses: seulement 1 bit d'erreur a été introduit, donc définir le seuil = 1
    et le décodage se terminera en 1 étape.

In [10]:
def demo(H, y):


    print("H:\n", H)
    r, n = H.shape
    w = sum(H[0,:] == 1)
    d = w // 2
    iteration = 1
    flipped = 1

    s = convertBinary(np.matmul(y, np.transpose(H)))
    print("\ny=mH^T + e:", y, "#ciphertext")
    print("Threshold T:", d)
    print("\n######### Starting the Bit-Flipping Algorithm... #########\n")

    print("s = yH^T:", s)
    print("sigmaJ:", np.matmul(s,H))
    while (np.count_nonzero(s) > 0 and flipped == 1):
        flipped = 0
        # syndrome weight
        T = 1

        for j in range(n):
            if (sum((s + H[:,j]) == 2)) >= T * d:
                print("FLIPPED position %d" % j)
                y[j] = y[j] ^ 1
                print("y:", y)
                s = convertBinary(np.matmul(y, np.transpose(H)))
                print("s = yH^T:", s)
                flipped = 1
                
        
        iteration += 1
        
        # syndrome
        s = np.matmul(y, np.transpose(H))
        s = convertBinary(s)

    print("Decrypted text:\n", y)
    if (sum(s == 1) == 0):
        return y[0: n-r]
    else:
        print("Cannot decode")
        return 0

In [11]:
# paramètres de code
n0 = 2
r = 5
wi = 3

N = 20
k = r // 2
################################## Paramètres de traitement ###################################

n = n0 * r
w = wi * n0
t = 2

##################################### Fonctions de test ####################################

# générer une matrice aléatoire (n, r, w) -QC-MDPC H
H = genQCMDPC(n, r, w)

# Générer la matrice génératrice G correspondante
G = genGenQCMDPC(H)
print("G:\n", G)
# générer un message aléatoire m de poids k
m = genRandomVector(r, k)

print("\nPlaintext m:", m)

# générer un vecteur d'erreur aléatoire e de poids t
e = genRandomVector(n, t)

print("\n e:", e)

# crypter le message m
y = encryptMcEliece(G, m, e)

# décrypter le texte chiffré
decryptedText = demo(H, y)

# vérifier si le décryptage est correct
decryptSuccess(m, decryptedText)


G:
 [[1 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 1 0 0]]

Plaintext m: [1 0 0 0 1]

 e: [0 1 0 0 0 0 0 0 0 1]
H:
 [[0 1 1 1 0 1 1 0 0 1]
 [0 0 1 1 1 1 1 1 0 0]
 [1 0 0 1 1 0 1 1 1 0]
 [1 1 0 0 1 0 0 1 1 1]
 [1 1 1 0 0 1 0 0 1 1]]

y=mH^T + e: [1 1 0 0 1 0 0 1 1 1] #ciphertext
Threshold T: 3

######### Starting the Bit-Flipping Algorithm... #########

s = yH^T: [0 0 0 0 0]
sigmaJ: [0 0 0 0 0 0 0 0 0 0]
Decrypted text:
 [1 1 0 0 1 0 0 1 1 1]
Decryption failure!


False